In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets

In [2]:
def get_batch(X, size):
    a = np.random.choice(len(X), size, replace=False)
    return X[a]      

In [3]:
class Autoencoder:
    def __init__(self, input_dim, hidden_dim, epoch=250, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.tanh(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
        
        self.x = x
        self.encoded = encoded
        self.decoded = decoded
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))
        self.train_op = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(num_samples):
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: [data[j]]})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, '../models/model.ckpt')
            self.saver.save(sess, '../models/model.ckpt')
            
    def test(self, data): 
        with tf.Session() as sess:
            self.saver.restore(sess, '../models/model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

    def train(self, data, batch_size=10):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                for j in range(500):
                    batch_data = get_batch(data, batch_size)
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: batch_data})
                if i % 10 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    self.saver.save(sess, '../models/model.ckpt')
            self.saver.save(sess, '../models/model.ckpt')

In [4]:
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

W0801 16:32:23.199314 4610823616 deprecation.py:506] From /Users/mattmann/git/buildout.python/python-3.7/lib/python3.7/site-packages/tensorflow_core/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


epoch 0: loss = 3.287008047103882
epoch 10: loss = 0.5802170038223267
epoch 20: loss = 0.30212658643722534
epoch 30: loss = 0.2772676348686218
epoch 40: loss = 0.34443092346191406
epoch 50: loss = 0.2815895080566406
epoch 60: loss = 0.31978023052215576
epoch 70: loss = 0.1632925420999527
epoch 80: loss = 0.25616973638534546
epoch 90: loss = 0.30626076459884644
epoch 100: loss = 0.28544068336486816
epoch 110: loss = 0.28550511598587036
epoch 120: loss = 0.44335800409317017
epoch 130: loss = 0.30442678928375244
epoch 140: loss = 0.31502965092658997
epoch 150: loss = 0.4268891215324402
epoch 160: loss = 0.325786828994751
epoch 170: loss = 0.2665727734565735
epoch 180: loss = 0.2218041568994522
epoch 190: loss = 0.3120783567428589
epoch 200: loss = 0.25270798802375793
epoch 210: loss = 0.268833190202713
epoch 220: loss = 0.33403274416923523
epoch 230: loss = 0.2214595377445221
epoch 240: loss = 0.2950868010520935
input [[8, 4, 6, 2]]
compressed [[0.4036218]]
reconstructed [[6.877247  2.816

array([[6.877247 , 2.816194 , 6.2037363, 2.2288456]], dtype=float32)